In [ ]:
%matplotlib inline

# ERCPy: Spectral unmixing demonstration notebook

#### by Martial Duchamp
Ernst Ruska-Centre for Microscopy and Spectroscopy with Electrons,
Research Centre Juelich, Germany,
m.duchamp@fz-juelich.de

### Absract:
The notebook demonstrates usage of the Vertex Component Anaylsis (VCA) spectral unmixing module of ERCPy package

## Preamble:

In [1]:
import hyperspy.api as hs
from scipy.linalg import pinv
import matplotlib.pyplot as plt
import hyperspy.api as hs
import os
import ercpy
import ercpy.utils as ut
import ercpy.eelsedx as eels
import numpy as np

#### Plotting outside the notebook to enable interactions with user:

In [ ]:
#%pylab qt

## Loading data:

In [2]:
path="/home/martial/Data/Esteem2/Myriam/Au_nws/unannealed/50C/"

In [3]:
file_name="EELS_aligned_croped.rpl"
specImg_ll = hs.load(path+file_name)

Loading as spectrum


#### Plotting:

In [ ]:
specImg_ll.plot()

In [4]:
#Crop your data in the eV range on which you want to perform the VCA decomposition
#If you enter float, it will be interpreted as eV, if you enter integer, it will be interpreted as pixel position
a_crop_min=1.0
b_crop_max=3.5
specImg_ll.crop_spectrum(a_crop_min,b_crop_max)

In [5]:
#Crop spatially, if needed
specImg_ll.crop(0,150,200)

In [6]:
#Create a new folder to store your data
vca_folder = path+"vca_folder"
if not os.path.exists(vca_folder): os.makedirs(vca_folder)

#### Estimation of the number of components you should keep (or the dimension of the subspace for the projection of the data)

In [7]:
specImg_ll.decomposition('svd', center_matrix="variables")


Performing decomposition analysis
Undoing data pre-treatments


In [8]:
#Remove negative values of the data
specImg_ll.data[specImg_ll.data < 0.] = 0.

In [9]:
#You can use the following function to get an idea how many dimensions you should keep on the subspace
nbr_compo=ercpy.eelsedx.Nbr_compotokeep(specImg_ll, 99.99)
nbr_compo

213

In [10]:
#Or define it by looking at the screeplot (variance)
specImg_ll.plot_explained_variance_ratio()

In [11]:
nbr_compo=3

In [13]:
specImg_ll.change_dtype("float64")

#### VCA Decomposition

In [14]:
Ae=ercpy.eelsedx.VCA_decomposition(specImg_ll, nbr_compo, True, True, True)

/usr/local/lib/python2.7/dist-packages/ercpy-0.1.0_dev-py2.7.egg/ercpy/eelsedx.py:55: RuntimeWarning: divide by zero encountered in divide


LinAlgError: SVD did not converge

##### Plot and save Abundance maps / linescan

In [ ]:
if np.size(specImg_cl.data.shape)==3:
    Ae3D = Ae.reshape(specImg_cl.data.shape[0],specImg_cl.data.shape[1],nbr_compo)
    i = 0
    while i<nbr_compo:
        fig, ax = plt.subplots()
        im = ax.imshow(Ae3D[:,:,i], vmin=abs(Ae3D[:,:,i]).min(), vmax=abs(Ae3D[:,:,i]).max()) #cmap=cm.RdBu, 
        im.set_interpolation('bilinear')
        cb = fig.colorbar(im, ax=ax)
        fig.savefig(path+'abundance_'+str(nbr_compo)+'_'+str(i)+'.png')
        i += 1
    
    #Save the maps in a rpl file to be imported back in DM for example
    Ae3D_maps = specImg_cl.deepcopy()
    Ae3D_maps.crop(2,0,nbr_compo)
    Ae3D_maps.data=Ae3D
    Ae3D_maps.save("abundance_maps_"+str(nbr_compo)+"_compo.rpl")

In [ ]:
if np.size(specImg_cl.data.shape)==2:
    i = 0
    X_scale = np.empty(Ae[:,0].size,dtype="float32")
    w=0

    scaling = 0.02

    while w < Ae[:,0].size:
        X_scale[w]=w
        w +=1
    
    fig, ax = plt.subplots()
    while i<nbr_compo:
        plt.plot( X_scale[:], Ae[:,i]-scaling*i, linewidth=2.0, label=str(i))
        plt.axhline(y=-scaling*i, xmin=0, xmax=1, hold=None)
        i += 1

    plt.legend( loc='upper right', numpoints = 1 )
    plt.show()
    savefig(path+'abundance_'+str(nbr_compo)+'.png')
    np.savetxt(path+'abundance_'+str(nbr_compo)+'.txt', Ae)

##### Plot and save endmembers profiles

In [ ]:
s_est = np.dot(np.linalg.pinv(Ae),ut.hspy_to_2Dnp(specImg_cl))
X_scale = np.empty(s_est[0,:].size,dtype="float32")
w=0

scaling = 1

while w < s_est[0,:].size:
    X_scale[w]=w*specImg_cl.axes_manager[2].scale+specImg_cl.axes_manager[2].offset
    w +=1

i = 0
fig = plt.figure()
while i<nbr_compo:
    plt.plot(X_scale[:], (s_est[i,:]-s_est[i,:].min())/(s_est[i,:].max()-s_est[i,:].min())-i*scaling, linewidth=2.0, label=str(i))
    plt.axhline(y=-scaling*i, xmin=0, xmax=1, hold=None)
    i = i +1
#plt.yscale('log')
plt.legend( loc='upper right', numpoints = 1 )
plt.show()
fig.savefig(path+'endmembers_'+str(nbr_compo)+'.png')
np.savetxt(path+'endmembers_'+str(nbr_compo)+'.txt', s_est)

In [ ]:
ercpy.utils.remove_output("VCA_v1.5ipynb)